In [1]:
# importing pandas

import pandas as pd

In [2]:
# importing data

df = pd.read_csv(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 62_Find the Duplicate Clicks\data.csv')

df.head()

,Installation Date,DeviceID,EventName,IDFA,EventFireDate
0,11/28/2015 17:57,2,Offer - Use at Location,1,11/30/2015 3:17:04
1,11/28/2015 17:57,2,Offer - Use at Location,1,12/6/2015 17:22:00
2,11/28/2015 17:57,2,Offer - Use at Location,1,12/6/2015 17:22:02
3,11/28/2015 17:57,2,Offer - Use at Location,1,1/30/2016 6:34:19
4,11/28/2015 17:57,2,Offer - Use at Location,1,1/30/2016 6:34:25


In [3]:
# formatting event fire date

df['EventFireDate'] = pd.to_datetime(df['EventFireDate'])

df.head()

,Installation Date,DeviceID,EventName,IDFA,EventFireDate
0,11/28/2015 17:57,2,Offer - Use at Location,1,2015-11-30 03:17:04
1,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:00
2,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:02
3,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:19
4,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:25


In [4]:
# sorting by device id and event fire date

df_sorted = df.sort_values(by=['DeviceID', 'EventFireDate'])

df_sorted.head()

,Installation Date,DeviceID,EventName,IDFA,EventFireDate
0,11/28/2015 17:57,2,Offer - Use at Location,1,2015-11-30 03:17:04
1,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:00
2,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:02
3,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:19
4,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:25


In [5]:
df_sorted.dtypes

Installation Date            object
DeviceID                      int64
EventName                    object
IDFA                          int64
EventFireDate        datetime64[ns]
dtype: object

In [6]:
# creating multi-row function to find elasped time

def elapsed_time(group):
    
    previous_time = None
    
    result = []
    
    for index, row in group.iterrows():
        
        current_time = row['EventFireDate']
        
        if previous_time is None:  
            result.append(pd.NA)
            
        else:
            elapsed = (current_time - previous_time).total_seconds()
            result.append(elapsed)
    
        previous_time = row['EventFireDate']
        
    group['ElapsedTime'] = result
    
    return group

In [7]:
# applying 

df_sorted = df_sorted.groupby('DeviceID').apply(elapsed_time).reset_index(drop = True)

In [8]:
df_sorted.head()

,Installation Date,DeviceID,EventName,IDFA,EventFireDate,ElapsedTime
0,11/28/2015 17:57,2,Offer - Use at Location,1,2015-11-30 03:17:04,<NA>
1,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:00,569096.0
2,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:02,2.0
3,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:19,4713137.0
4,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:25,6.0


In [16]:
# flagging if elasped time is greater than 30 sec

df_sorted['duplicate_click'] = df_sorted['ElapsedTime'].apply(lambda x: 1 if pd.notna(x) and x <= 30 else 0)

df_sorted.head()

,Installation Date,DeviceID,EventName,IDFA,EventFireDate,ElapsedTime,duplicate_click
0,11/28/2015 17:57,2,Offer - Use at Location,1,2015-11-30 03:17:04,<NA>,0
1,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:00,569096.0,0
2,11/28/2015 17:57,2,Offer - Use at Location,1,2015-12-06 17:22:02,2.0,1
3,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:19,4713137.0,0
4,11/28/2015 17:57,2,Offer - Use at Location,1,2016-01-30 06:34:25,6.0,1


In [17]:
# checking answer against alteryx

df_sorted['duplicate_click'].sum()

7